In [36]:
#CONDA ENV base_conda (python 3.9.7)
#Import packages
#---------------------------------------
import sys
import os
import glob
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm


#Import your modules
#---------------------------------------
import te_rna_f as ter
sys.path.insert(1, '/cndd3/dburrows/CODE/admin_tools/')
from admin_tools import admin_functions as adm

# Define paths
#----------------------------------------------------------------------
l_code = '/Users/dominicburrows/Dropbox/PhD/Analysis/my_scripts/GitHub/'
l_data = '/Users/dominicburrows/Dropbox/PhD/analysis/Project/'
l_fig = '/Users/dominicburrows/Dropbox/PhD/figures/'

s_code = '/cndd3/dburrows/CODE/'
s_data = '/cndd3/dburrows/DATA/'
s_fig = '/cndd3/dburrows/FIGS/'

%load_ext autoreload
sys.version

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:23:11) \n[GCC 9.4.0]'

In [12]:
in_path = '/cndd3/dburrows/DATA/te/rna/PE.bam/'
out_path='/cndd/dburrows/DATA/te/rna/PE.counts/multimap/'

dir_list = [os.path.basename(i) for i in glob.glob(in_path + '*Samp*')]
x=0
d=dir_list[0]

# remove read 1,non-canonical chrs and filter by L1HS loci
run = f"""
samtools view -b -F 0x40 {in_path}/{d}/Aligned.sortedByCoord.out.bam > {in_path}/{d}/Aligned.sortedByCoord.out.r2.bam
samtools index {in_path}/{d}/Aligned.sortedByCoord.out.r2.bam
samtools view -o {in_path}/{d}/Aligned.sortedByCoord.out.r2.canon.bam {in_path}/{d}/Aligned.sortedByCoord.out.r2.bam $(seq 1 22 | sed 's/^/chr/') chrX chrY
samtools index {in_path}/{d}/Aligned.sortedByCoord.out.r2.canon.bam
bedtools intersect -s -wa -a {in_path}/{d}/Aligned.sortedByCoord.out.r2.canon.bam -b /cndd3/dburrows/DATA/annotations/rmsk/rmsk.hg38.filt-5ptrim.merge.L1HS_all.canon.sorted.bed \
# > {out_path}/{d}/L1HS_all.bam
""" 

get_ipython().run_cell_magic('bash', '', run)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

ERROR: Database file /cndd3/dburrows/DATA/te/rna/PE.bam//Sample_5643_GLU/Aligned.sortedByCoord.out.r2.canon.bam contains chromosome chr15, but the query file does not.
       Please rerun with the -g option for a genome file.
       See documentation for details.


CalledProcessError: Command 'b"\nsamtools view -b -F 0x40 /cndd3/dburrows/DATA/te/rna/PE.bam//Sample_5643_GLU/Aligned.sortedByCoord.out.bam > /cndd3/dburrows/DATA/te/rna/PE.bam//Sample_5643_GLU/Aligned.sortedByCoord.out.r2.bam\nsamtools index /cndd3/dburrows/DATA/te/rna/PE.bam//Sample_5643_GLU/Aligned.sortedByCoord.out.r2.bam\nsamtools view -o /cndd3/dburrows/DATA/te/rna/PE.bam//Sample_5643_GLU/Aligned.sortedByCoord.out.r2.canon.bam /cndd3/dburrows/DATA/te/rna/PE.bam//Sample_5643_GLU/Aligned.sortedByCoord.out.r2.bam $(seq 1 22 | sed 's/^/chr/') chrX chrY\nsamtools index /cndd3/dburrows/DATA/te/rna/PE.bam//Sample_5643_GLU/Aligned.sortedByCoord.out.r2.canon.bam\nbedtools intersect -s -sorted -wb -b /cndd3/dburrows/DATA/te/rna/PE.bam//Sample_5643_GLU/Aligned.sortedByCoord.out.r2.canon.bam -a /cndd3/dburrows/DATA/annotations/rmsk/rmsk.hg38.filt-5ptrim.merge.L1HS_all.canon.sorted.bed # > /cndd/dburrows/DATA/te/rna/PE.counts/multimap//Sample_5643_GLU/L1HS_all.bam\n"' returned non-zero exit s

In [13]:
in_path = '/cndd3/dburrows/DATA/te/rna/PE.bam/'
out_path='/cndd/dburrows/DATA/te/rna/PE.counts/multimap/'

dir_list = [os.path.basename(i) for i in glob.glob(in_path + '*Samp*')]
x=0
d=dir_list[0]

# remove read 1,non-canonical chrs and filter by L1HS loci
run = f"""
bedtools intersect -s -wb -b {in_path}/{d}/Aligned.sortedByCoord.out.r2.canon.bam -a /cndd3/dburrows/DATA/annotations/rmsk/rmsk.hg38.filt-5ptrim.merge.L1HS_all.canon.sorted.bed \
 > {out_path}/{d}/L1HS_all.bam
""" 

get_ipython().run_cell_magic('bash', '', run)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
bedtools intersect -s -wb -b /cndd3/dburrows/DATA/te/rna/PE.bam/Sample_5643_GLU/Aligned.sortedByCoord.out.r2.canon.bam -a /cndd3/dburrows/DATA/annotations/rmsk/rmsk.hg38.filt-5ptrim.merge.L1HS_all.canon.sorted.bed > ./L1HS_all.bam



In [232]:
#CONDA ENV base (python 3.9.12)
#Import packages
#---------------------------------------
import json
import pandas as pd
import sys
import numpy as np
import glob
import pysam

#Import your modules
#---------------------------------------
sys.path.insert(1, '/cndd3/dburrows/CODE/te_rna_ageing/')
import te_rna_f as te
sys.path.insert(1, '/cndd3/dburrows/CODE/admin_tools/')
from admin_tools import admin_functions as adm
#os.chdir('/cndd3/dburrows/CODE/te_rna_ageing/snakemake/workflow_PE-GLU.ATEM/')
os.chdir('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/Sample_1134_GLU/')
! ls -ltrh

# ! mv plus.5pfilt_withsplice.bam plus.5pfilt.withr1.bam
# ! mv minus.5pfilt_withsplice.bam minus.5pfilt.withr1.bam
# ! samtools index plus.5pfilt.withr1.bam
# ! samtools index minus.5pfilt.withr1.bam

# ! samtools view -b -F 0x40 plus.5pfilt.withr1.bam > plus.5pfilt.bam
# ! samtools view -b -F 0x40 minus.5pfilt.withr1.bam > minus.5pfilt.bam
# ! samtools index plus.5pfilt.bam
# ! samtools index minus.5pfilt.bam

# ! samtools view plus.5pfilt.bam | cut -f -9 > plus.5pfilt.metadata.txt
# ! samtools view minus.5pfilt.bam | cut -f -9 > minus.5pfilt.metadata.txt

! samtools view -h plus.5pfilt.bam | awk 'NR==FNR{{a[$1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$6"\t"$7"\t"$8"\t"$9];next}} !($1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$6"\t"$7"\t"$8"\t"$9 in a)' plus.5pfilt-notin.metadata.txt -> plus.5pfilt-tss.sam
! samtools view -h minus.5pfilt.bam | awk 'NR==FNR{{a[$1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$6"\t"$7"\t"$8"\t"$9];next}} !($1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$6"\t"$7"\t"$8"\t"$9 in a)' minus.5pfilt-notin.metadata.txt -> minus.5pfilt-tss.sam


! samtools view -S -b plus.5pfilt-tss.sam > plus.5pfilt-tss.bam
! samtools view -S -b minus.5pfilt-tss.sam > minus.5pfilt-tss.bam
! samtools merge -f 5pfilt-tss.bam plus.5pfilt-tss.bam minus.5pfilt-tss.bam
! samtools index 5pfilt-tss.bam

total 29G
-rw-r--r-- 1 dburrows cogsci-cndd-user   10 Jan  9 17:56 total_reads.txt
-rw-r--r-- 1 dburrows cogsci-cndd-user  51G Jan  9 18:15 plus.bam
-rw-r--r-- 1 dburrows cogsci-cndd-user  51G Jan  9 18:15 minus.bam
-rw-r--r-- 1 dburrows cogsci-cndd-user 2.0M Jan  9 19:28 plus.5pfilt_withsplice.bam.bai
-rw-r--r-- 1 dburrows cogsci-cndd-user 2.0M Jan  9 19:51 minus.5pfilt_withsplice.bam.bai
-rw-r--r-- 1 dburrows cogsci-cndd-user 177M Jan  9 19:51 plus.5pfilt.withr1.bam
-rw-r--r-- 1 dburrows cogsci-cndd-user 177M Jan  9 19:51 minus.5pfilt.withr1.bam
-rw-r--r-- 1 dburrows cogsci-cndd-user  51M Jan  9 19:58 ATEM_counts.csv
-rw-r--r-- 1 dburrows cogsci-cndd-user 2.0M Jan 10 03:45 plus.5pfilt.withr1.bam.bai
-rw-r--r-- 1 dburrows cogsci-cndd-user 2.0M Jan 10 03:46 minus.5pfilt.withr1.bam.bai
-rw-r--r-- 1 dburrows cogsci-cndd-user  87M Jan 10 03:47 plus.5pfilt.bam
-rw-r--r-- 1 dburrows cogsci-cndd-user  82M Jan 10 03:47 minus.5pfilt.bam
-rw-r--r-- 1 dburrows cogsci-cndd-user 1.9M Jan 10 03:47 

In [257]:
pl_umi

,0,1,2,3,4,5,6,7,8
3,A00132:417:HKVTWDSX3:3:2323:15709:9032,419,chr1,120888,1,45M44951N50M,=,768487,647698
4,A00132:417:HKVTWDSX3:4:1405:30906:9893,419,chr1,124469,0,77M,=,124469,77
5,A00132:417:HKVTWDSX3:4:1177:26756:10598,419,chr1,141574,0,97M,=,141617,143
6,A00132:417:HKVTWDSX3:4:1177:27326:13150,419,chr1,141574,0,97M,=,141617,143
7,A00132:417:HKVTWDSX3:4:1177:27145:13808,419,chr1,141574,0,97M,=,141617,143
...,...,...,...,...,...,...,...,...,...
1863591,A00132:417:HKVTWDSX3:4:1467:26467:35775,419,chrUn_KI270742v1,87216,1,2S92M,=,87223,106
1863592,A00132:417:HKVTWDSX3:3:1212:27453:25238,163,chrUn_KI270742v1,87276,3,97M,=,87461,285
1863594,A00132:417:HKVTWDSX3:3:2602:4010:30170,419,chrUn_KI270742v1,95066,1,97M,=,95107,140
1863596,A00132:417:HKVTWDSX3:4:2454:15148:28855,163,chrUn_GL000218v1,72377,255,95M,=,72387,109


In [258]:
bam_pl

,Chromosome,Start,End,Strand,Flag,UMI
0,chr1,31405,31502,+,419,A00132:417:HKVTWDSX3:3:2551:3721:30076
1,chr1,51632,51717,+,163,A00132:417:HKVTWDSX3:3:2105:30816:26929
2,chr1,111095,111166,+,419,A00132:417:HKVTWDSX3:4:2218:14488:27398
3,chr1,120887,165933,+,419,A00132:417:HKVTWDSX3:3:2323:15709:9032
4,chr1,124468,124545,+,419,A00132:417:HKVTWDSX3:4:1405:30906:9893
...,...,...,...,...,...,...
1863595,chrUn_KI270742v1,102192,102289,+,163,A00132:417:HKVTWDSX3:3:1238:30924:19382
1863596,chrUn_GL000218v1,72376,72471,+,163,A00132:417:HKVTWDSX3:4:2454:15148:28855
1863597,chrUn_GL000218v1,72377,72471,+,163,A00132:417:HKVTWDSX3:3:2304:13648:5337
1863598,chrUn_GL000218v1,90526,90622,+,163,A00132:417:HKVTWDSX3:4:1316:30246:8876


In [256]:
len(pl_umi) + len(bam_ll[0])

1863600

In [ ]:
! samtools view -h plus.5pfilt.bam | awk 'NR==FNR{{a[$1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$6"\t"$7"\t"$8"\t"$9];next}} !($1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$6"\t"$7"\t"$8"\t"$9 in a)' plus.5pfilt-notin.metadata.txt -> plus.5pfilt-tss.sam
! samtools view -h minus.5pfilt.bam | awk 'NR==FNR{{a[$1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$6"\t"$7"\t"$8"\t"$9];next}} !($1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$6"\t"$7"\t"$8"\t"$9 in a)' minus.5pfilt-notin.metadata.txt -> minus.5pfilt-tss.sam


! samtools view -S -b plus.5pfilt-tss.sam > plus.5pfilt-tss.bam
! samtools view -S -b minus.5pfilt-tss.sam > minus.5pfilt-tss.bam
! samtools merge -f 5pfilt-tss.bam plus.5pfilt-tss.bam minus.5pfilt-tss.bam
! samtools index 5pfilt-tss.bam


        plus=config['out_path']+"/{ID}/plus.5pfilt.bam",
        minus=config['out_path']+"/{ID}/minus.5pfilt.bam",
        meta_pl_notin=config['out_path']+"/{ID}/plus.5pfilt-notin.metadata.txt",
        meta_mi_notin=config['out_path']+"/{ID}/minus.5pfilt-notin.metadata.txt"

    output:
        plus_sam=config['out_path']+"/{ID}/plus.5pfilt-tss.sam",
        minus_sam=config['out_path']+"/{ID}/minus.5pfilt-tss.sam"

    shell:
        """
        samtools view -h {input.plus} | awk 'NR==FNR{{a[$1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$6"\t"$7"\t"$8"\t"$9];next}} !($1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$6"\t"$7"\t"$8"\t"$9 in a)' {input.meta_pl_notin} -> {output.plus_sam}
        samtools view -h {input.minus} | awk 'NR==FNR{{a[$1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$6"\t"$7"\t"$8"\t"$9];next}} !($1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$6"\t"$7"\t"$8"\t"$9 in a)' {input.meta_mi_notin} -> {output.minus_sam}
        """


#return sam file with only 5p filt at tss reads
#============================================
rule filter_bam_by_ATEM:
#============================================
    input:
        plus=config['out_path']+"/{ID}/plus.5pfilt.bam",
        minus=config['out_path']+"/{ID}/minus.5pfilt.bam",
        meta_pl_notin=config['out_path']+"/{ID}/plus.5pfilt-notin.metadata.txt",
        meta_mi_notin=config['out_path']+"/{ID}/minus.5pfilt-notin.metadata.txt"

    output:
        plus_sam=config['out_path']+"/{ID}/plus.5pfilt-tss.sam",
        minus_sam=config['out_path']+"/{ID}/minus.5pfilt-tss.sam"

    shell:
        """
        samtools view -h {input.plus} | awk 'NR==FNR{{a[$1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$6"\t"$7"\t"$8"\t"$9];next}} !($1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$6"\t"$7"\t"$8"\t"$9 in a)' {input.meta_pl_notin} -> {output.plus_sam}
        samtools view -h {input.minus} | awk 'NR==FNR{{a[$1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$6"\t"$7"\t"$8"\t"$9];next}} !($1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$6"\t"$7"\t"$8"\t"$9 in a)' {input.meta_mi_notin} -> {output.minus_sam}
        """



In [ ]:
/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/Sample_5606_GLU/

In [22]:
#CONDA ENV base (python 3.9.12)
#Import packages
#---------------------------------------
import json
import pandas as pd
import sys
import numpy as np
import glob
import pysam

#Import your modules
#---------------------------------------
sys.path.insert(1, '/cndd3/dburrows/CODE/te_rna_ageing/')
import te_rna_f as te
sys.path.insert(1, '/cndd3/dburrows/CODE/admin_tools/')
from admin_tools import admin_functions as adm
#os.chdir('/cndd3/dburrows/CODE/te_rna_ageing/snakemake/workflow_PE-GLU.ATEM/')
os.chdir('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/Sample_5606_GLU/')

! ls -ltrh
# ! mv plus.5pfilt_withsplice.bam plus.5pfilt.withr1.bam
# ! mv minus.5pfilt_withsplice.bam minus.5pfilt.withr1.bam
# ! samtools index plus.5pfilt.withr1.bam
# ! samtools index minus.5pfilt.withr1.bam

# ! samtools view -b -F 0x40 plus.5pfilt.withr1.bam > plus.5pfilt.bam
# ! samtools view -b -F 0x40 minus.5pfilt.withr1.bam > minus.5pfilt.bam
# ! samtools index plus.5pfilt.bam
# ! samtools index minus.5pfilt.bam

# ! samtools view plus.5pfilt.bam | cut -f -9 > plus.5pfilt.metadata.txt
# ! samtools view minus.5pfilt.bam | cut -f -9 > minus.5pfilt.metadata.txt


#! head plus.5pfilt-notin.metadata.txt

# ! wc -l plus.5pfilt.metadata.txt
! samtools view plus.5pfilt.bam | wc -l
! wc -l plus.5pfilt.metadata.txt 
! wc -l plus.5pfilt-notin.metadata.txt 
! samtools view plus.5pfilt-tss.bam | wc -l

total 27G
-rw-r--r-- 1 dburrows cogsci-cndd-user  50G Jan 10 08:33 plus.bam
-rw-r--r-- 1 dburrows cogsci-cndd-user  50G Jan 10 08:33 minus.bam
-rw-r--r-- 1 dburrows cogsci-cndd-user 2.0M Jan 10 09:18 plus.5pfilt_withsplice.bam.bai
-rw-r--r-- 1 dburrows cogsci-cndd-user   10 Jan 10 09:38 total_reads.txt
-rw-r--r-- 1 dburrows cogsci-cndd-user 2.0M Jan 10 09:38 minus.5pfilt_withsplice.bam.bai
-rw-r--r-- 1 dburrows cogsci-cndd-user 157M Jan 10 09:38 plus.5pfilt_withsplice.bam
-rw-r--r-- 1 dburrows cogsci-cndd-user 152M Jan 10 09:38 minus.5pfilt_withsplice.bam
-rw-r--r-- 1 dburrows cogsci-cndd-user 2.8M Jan 10 09:39 plus.5pfilt.withr1.bam.bai
-rw-r--r-- 1 dburrows cogsci-cndd-user 2.8M Jan 10 09:39 minus.5pfilt.withr1.bam.bai
-rw-r--r-- 1 dburrows cogsci-cndd-user  62M Jan 10 09:39 plus.5pfilt.withr1.bam
-rw-r--r-- 1 dburrows cogsci-cndd-user  60M Jan 10 09:39 minus.5pfilt.withr1.bam
-rw-r--r-- 1 dburrows cogsci-cndd-user 2.4M Jan 10 09:39 plus.5pfilt.bam.bai
-rw-r--r-- 1 dburrows cogsci-cn

In [260]:
pl_umi

,0,1,2,3,4,5,6,7,8
3,A00132:417:HKVTWDSX3:3:2323:15709:9032,419,chr1,120888,1,45M44951N50M,=,768487,647698
4,A00132:417:HKVTWDSX3:4:1405:30906:9893,419,chr1,124469,0,77M,=,124469,77
5,A00132:417:HKVTWDSX3:4:1177:26756:10598,419,chr1,141574,0,97M,=,141617,143
6,A00132:417:HKVTWDSX3:4:1177:27326:13150,419,chr1,141574,0,97M,=,141617,143
7,A00132:417:HKVTWDSX3:4:1177:27145:13808,419,chr1,141574,0,97M,=,141617,143
...,...,...,...,...,...,...,...,...,...
1863591,A00132:417:HKVTWDSX3:4:1467:26467:35775,419,chrUn_KI270742v1,87216,1,2S92M,=,87223,106
1863592,A00132:417:HKVTWDSX3:3:1212:27453:25238,163,chrUn_KI270742v1,87276,3,97M,=,87461,285
1863594,A00132:417:HKVTWDSX3:3:2602:4010:30170,419,chrUn_KI270742v1,95066,1,97M,=,95107,140
1863596,A00132:417:HKVTWDSX3:4:2454:15148:28855,163,chrUn_GL000218v1,72377,255,95M,=,72387,109


In [31]:
bam_pl='plus.5pfilt.bam'
bam_mi='minus.5pfilt.bam'
meta_pl='plus.5pfilt.metadata.txt'
meta_mi='minus.5pfilt.metadata.txt'
n_reads='total_reads.txt'

In [158]:
bam_pl = te.pysam_subset(path+bam_pl) 
bam_mi = te.pysam_subset(path+bam_mi) 
meta_bpl = pd.read_csv(path+meta_pl, sep='\t', header=None)
meta_bmi = pd.read_csv(path+meta_mi, sep='\t', header=None)

[W::hts_idx_load3] The index file is older than the data file: /cndd/dburrows/DATA/te/rna/PE.counts/ATEM/Sample_HCT16HCQA020_GLU/plus.5pfilt.bam.bai


In [159]:
bam_pl

,Chromosome,Start,End,Strand,Flag,UMI
0,chr1,110980,111034,+,419,A00132:417:HKVTWDSX3:3:1175:10610:2675:read2
1,chr1,141566,141663,+,419,A00132:417:HKVTWDSX3:3:2152:8486:31845:read2
2,chr1,147525,147601,+,419,A00132:417:HKVTWDSX3:4:1577:3794:36182:read2
3,chr1,147525,147601,+,419,A00132:417:HKVTWDSX3:4:1577:3794:36182:read2
4,chr1,147641,147665,+,419,A00132:417:HKVTWDSX3:3:1213:7084:25692:read2
...,...,...,...,...,...,...
816159,chrUn_KI270742v1,78639,78734,+,163,A00132:417:HKVTWDSX3:3:2452:10384:13683:read2
816160,chrUn_KI270742v1,78639,78734,+,163,A00132:417:HKVTWDSX3:4:2425:6470:20087:read2
816161,chrUn_KI270742v1,78683,78778,+,163,A00132:417:HKVTWDSX3:4:1678:13485:13448:read2
816162,chrUn_KI270742v1,83574,83663,+,419,A00132:417:HKVTWDSX3:4:1369:32108:21089:read2


In [161]:
meta_bpl

,0,1,2,3,4,5,6,7,8
0,A00132:417:HKVTWDSX3:4:2462:26377:9095,355,chr1,26653,1,3S96M,=,26684,128
1,A00132:417:HKVTWDSX3:4:2634:11062:18991,99,chr1,26653,1,3S96M,=,26684,128
2,A00132:417:HKVTWDSX3:4:1144:25789:2253,99,chr1,26751,0,97M,=,26852,198
3,A00132:417:HKVTWDSX3:4:1303:19651:17769,355,chr1,26772,1,2S98M,=,26886,211
4,A00132:417:HKVTWDSX3:4:2414:18548:26694,355,chr1,26772,1,2S98M,=,26886,211
...,...,...,...,...,...,...,...,...,...
1413085,A00132:417:HKVTWDSX3:3:2133:27932:18333,99,chrUn_GL000218v1,77507,255,100M,=,77520,110
1413086,A00132:417:HKVTWDSX3:3:1456:15058:23093,355,chrUn_GL000218v1,90521,1,99M,=,90527,101
1413087,A00132:417:HKVTWDSX3:3:1456:15356:23735,355,chrUn_GL000218v1,90521,1,99M,=,90527,101
1413088,A00132:417:HKVTWDSX3:3:1456:15799:25285,355,chrUn_GL000218v1,90521,1,99M,=,90527,101


In [198]:
path = '/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/Sample_1134_GLU/'

'/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/Sample_HCT16HCQA020_GLU/'

In [219]:
plus.5pfilt.bam

In [265]:
bam_pl

,Chromosome,Start,End,Strand,Flag,UMI
0,chr1,31405,31502,+,419,A00132:417:HKVTWDSX3:3:2551:3721:30076
1,chr1,51632,51717,+,163,A00132:417:HKVTWDSX3:3:2105:30816:26929
2,chr1,111095,111166,+,419,A00132:417:HKVTWDSX3:4:2218:14488:27398
3,chr1,120887,165933,+,419,A00132:417:HKVTWDSX3:3:2323:15709:9032
4,chr1,124468,124545,+,419,A00132:417:HKVTWDSX3:4:1405:30906:9893
...,...,...,...,...,...,...
1863595,chrUn_KI270742v1,102192,102289,+,163,A00132:417:HKVTWDSX3:3:1238:30924:19382
1863596,chrUn_GL000218v1,72376,72471,+,163,A00132:417:HKVTWDSX3:4:2454:15148:28855
1863597,chrUn_GL000218v1,72377,72471,+,163,A00132:417:HKVTWDSX3:3:2304:13648:5337
1863598,chrUn_GL000218v1,90526,90622,+,163,A00132:417:HKVTWDSX3:4:1316:30246:8876


In [32]:
os.chdir('/cndd3/dburrows/CODE/te_rna_ageing/snakemake/workflow_PE-GLU.ATEM/')
path = '/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/Sample_5606_GLU/'

#Read in required files for filtering
js = json.load(open(glob.glob('*config*')[0])) #CHANGE TO MAKE MORE FLEXIBLE?
bed_pl = pd.read_csv(js['bed_plus_path'], sep='\t', header=None)
bed_pl.columns =['Chromosome', 'Start', 'End', 'Strand', 'gene_id', 'family_id', 'class_id', 'length', 'full_Start', 'full_End']
bed_pl = bed_pl.drop(columns=['length'])
bed_mi = pd.read_csv(js['bed_minus_path'],sep='\t', header=None)
bed_mi.columns =['Chromosome', 'Start', 'End', 'Strand', 'gene_id', 'family_id', 'class_id', 'length', 'full_Start', 'full_End']
bed_mi = bed_mi.drop(columns=['length'])


# bam_pl = pr.read_bam(snakemake.input.bam_pl, as_df=True) 
# bam_mi = pr.read_bam(snakemake.input.bam_mi, as_df=True) 
bam_pl = te.pysam_subset(path+bam_pl) 
bam_mi = te.pysam_subset(path+bam_mi) 


#Swap Start + End for minus strand
bam_mi['Start'], bam_mi['End'] = bam_mi['End'], bam_mi['Start']

#File checks
assert sum(bam_pl['Strand'] == '+') == len(bam_pl), 'Some non plus strands assigned to plus bam'
assert sum(bam_mi['Strand'] == '-') == len(bam_mi), 'Some non minus strands assigned to minus bam'
assert sum(bed_pl['Strand'] == '+') == len(bed_pl), 'Some non plus strands assigned to plus bed'
assert sum(bed_mi['Strand'] == '-') == len(bed_mi), 'Some non minus strands assigned to minus bed'

#Define + and - strand files
pl_pars = [bed_pl, bam_pl, meta_pl, 'plus'] 
mi_pars = [bed_mi, bam_mi, meta_mi, 'minus'] 
par_list = [pl_pars, mi_pars]

[W::hts_idx_load3] The index file is older than the data file: /cndd/dburrows/DATA/te/rna/PE.counts/ATEM/Sample_5606_GLU/plus.5pfilt.bam.bai


In [248]:
bam_pl

,Chromosome,Start,End,Strand,Flag,UMI
0,chr1,31405,31502,+,419,A00132:417:HKVTWDSX3:3:2551:3721:30076
1,chr1,51632,51717,+,163,A00132:417:HKVTWDSX3:3:2105:30816:26929
2,chr1,111095,111166,+,419,A00132:417:HKVTWDSX3:4:2218:14488:27398
3,chr1,120887,165933,+,419,A00132:417:HKVTWDSX3:3:2323:15709:9032
4,chr1,124468,124545,+,419,A00132:417:HKVTWDSX3:4:1405:30906:9893
...,...,...,...,...,...,...
1863595,chrUn_KI270742v1,102192,102289,+,163,A00132:417:HKVTWDSX3:3:1238:30924:19382
1863596,chrUn_GL000218v1,72376,72471,+,163,A00132:417:HKVTWDSX3:4:2454:15148:28855
1863597,chrUn_GL000218v1,72377,72471,+,163,A00132:417:HKVTWDSX3:3:2304:13648:5337
1863598,chrUn_GL000218v1,90526,90622,+,163,A00132:417:HKVTWDSX3:4:1316:30246:8876


In [266]:
pl_umi

,0,1,2,3,4,5,6,7,8
3,A00132:417:HKVTWDSX3:3:2323:15709:9032,419,chr1,120888,1,45M44951N50M,=,768487,647698
4,A00132:417:HKVTWDSX3:4:1405:30906:9893,419,chr1,124469,0,77M,=,124469,77
5,A00132:417:HKVTWDSX3:4:1177:26756:10598,419,chr1,141574,0,97M,=,141617,143
6,A00132:417:HKVTWDSX3:4:1177:27326:13150,419,chr1,141574,0,97M,=,141617,143
7,A00132:417:HKVTWDSX3:4:1177:27145:13808,419,chr1,141574,0,97M,=,141617,143
...,...,...,...,...,...,...,...,...,...
1863591,A00132:417:HKVTWDSX3:4:1467:26467:35775,419,chrUn_KI270742v1,87216,1,2S92M,=,87223,106
1863592,A00132:417:HKVTWDSX3:3:1212:27453:25238,163,chrUn_KI270742v1,87276,3,97M,=,87461,285
1863594,A00132:417:HKVTWDSX3:3:2602:4010:30170,419,chrUn_KI270742v1,95066,1,97M,=,95107,140
1863596,A00132:417:HKVTWDSX3:4:2454:15148:28855,163,chrUn_GL000218v1,72377,255,95M,=,72387,109


In [250]:
bam_ll[0].shape

(320260,)

In [33]:
pd.options.mode.chained_assignment = None  # default='warn'
count_df = pd.DataFrame() #empty count matrix
bam_ll = [[],[]] #empty list of lists to store curr_bam indices

# # Filter out reads that do not overlap with 5' portion of insertion
for x,par in enumerate(par_list):
    curr_bed = par[0]
    curr_bam = par[1]
#     curr_name = pd.read_csv(par[2], sep='\t', header=None)
#     assert len(curr_bam) == len(curr_name), 'Bam and metadata files not the same length'
#     curr_bam['UMI']=curr_name[0].values #Add UMI column to bam file

    #Loop through each chromosome
    chr_unq = np.unique(curr_bam['Chromosome'].values)
    for i,chr in enumerate(chr_unq):
        print('Aligning to chromosome ' + chr + ' for ' + par[3] + ' strand')
        
        #Slice bed/bam files by chromosome
        chr_bam = curr_bam[curr_bam['Chromosome'] == chr]
        chr_bed = curr_bed[curr_bed['Chromosome'] == chr]
        count_df, bam_ll[x] = te.five_prime_align(chr_bam, chr_bed, count_df, bam_ll[x])

#Add in CPMs as a column
total_reads = pd.read_csv(path+n_reads, sep=" ", header=None)[0].values[0] 
count_df['CPM'] = count_df['Count'].values / total_reads * 1000000 

Aligning to chromosome chr1 for plus strand
Aligning to chromosome chr10 for plus strand
Aligning to chromosome chr11 for plus strand
Aligning to chromosome chr11_KI270721v1_random for plus strand
Aligning to chromosome chr12 for plus strand
Aligning to chromosome chr13 for plus strand
Aligning to chromosome chr14 for plus strand
Aligning to chromosome chr14_GL000009v2_random for plus strand
Aligning to chromosome chr14_GL000194v1_random for plus strand
Aligning to chromosome chr14_KI270725v1_random for plus strand
Aligning to chromosome chr15 for plus strand
Aligning to chromosome chr15_KI270727v1_random for plus strand
Aligning to chromosome chr16 for plus strand
Aligning to chromosome chr16_KI270728v1_random for plus strand
Aligning to chromosome chr17 for plus strand
Aligning to chromosome chr18 for plus strand
Aligning to chromosome chr19 for plus strand
Aligning to chromosome chr1_KI270706v1_random for plus strand
Aligning to chromosome chr1_KI270712v1_random for plus strand
Alig

In [224]:
bam_ind

'5pfilt-tss_splitstrand.index.npy'

In [214]:
np.save(path+bam_ind,np.asarray(bam_ll, dtype=object))


In [228]:
# #Save counts matrix and bam indeces
# count_df.to_csv(snakemake.output.count_mat, sep='\t', index=False)
np.save(path+bam_ind,np.asarray(bam_ll, dtype=object))

#Make txt file of start sites to remove
if len(bam_ll[0]) > 0:
    pl_umi=pd.read_csv(path+meta_pl, sep='\t', header=None).iloc[np.setxor1d(np.arange(0,len(bam_pl)) , bam_ll[0].astype(int))]
elif len(bam_ll[0]) == 0:
    pl_umi=pd.read_csv(path+meta_pl, sep='\t', header=None)
if len(bam_ll[1]) > 0:
    mi_umi=pd.read_csv(path+meta_mi, sep='\t', header=None).iloc[np.setxor1d(np.arange(0,len(bam_mi)) , bam_ll[1].astype(int))]
elif len(bam_ll[1]) == 0:
    mi_umi=pd.read_csv(path+meta_mi, sep='\t', header=None)

np.savetxt(path+meta_pl_notin,  pl_umi, fmt='%s')
np.savetxt(path+meta_mi_notin,  mi_umi, fmt='%s')

In [267]:
pl_umi

,0,1,2,3,4,5,6,7,8
3,A00132:417:HKVTWDSX3:3:2323:15709:9032,419,chr1,120888,1,45M44951N50M,=,768487,647698
4,A00132:417:HKVTWDSX3:4:1405:30906:9893,419,chr1,124469,0,77M,=,124469,77
5,A00132:417:HKVTWDSX3:4:1177:26756:10598,419,chr1,141574,0,97M,=,141617,143
6,A00132:417:HKVTWDSX3:4:1177:27326:13150,419,chr1,141574,0,97M,=,141617,143
7,A00132:417:HKVTWDSX3:4:1177:27145:13808,419,chr1,141574,0,97M,=,141617,143
...,...,...,...,...,...,...,...,...,...
1863591,A00132:417:HKVTWDSX3:4:1467:26467:35775,419,chrUn_KI270742v1,87216,1,2S92M,=,87223,106
1863592,A00132:417:HKVTWDSX3:3:1212:27453:25238,163,chrUn_KI270742v1,87276,3,97M,=,87461,285
1863594,A00132:417:HKVTWDSX3:3:2602:4010:30170,419,chrUn_KI270742v1,95066,1,97M,=,95107,140
1863596,A00132:417:HKVTWDSX3:4:2454:15148:28855,163,chrUn_GL000218v1,72377,255,95M,=,72387,109


In [264]:
path

'/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/Sample_1134_GLU/'

In [254]:
bam_pl

,Chromosome,Start,End,Strand,Flag,UMI
0,chr1,31405,31502,+,419,A00132:417:HKVTWDSX3:3:2551:3721:30076
1,chr1,51632,51717,+,163,A00132:417:HKVTWDSX3:3:2105:30816:26929
2,chr1,111095,111166,+,419,A00132:417:HKVTWDSX3:4:2218:14488:27398
3,chr1,120887,165933,+,419,A00132:417:HKVTWDSX3:3:2323:15709:9032
4,chr1,124468,124545,+,419,A00132:417:HKVTWDSX3:4:1405:30906:9893
...,...,...,...,...,...,...
1863595,chrUn_KI270742v1,102192,102289,+,163,A00132:417:HKVTWDSX3:3:1238:30924:19382
1863596,chrUn_GL000218v1,72376,72471,+,163,A00132:417:HKVTWDSX3:4:2454:15148:28855
1863597,chrUn_GL000218v1,72377,72471,+,163,A00132:417:HKVTWDSX3:3:2304:13648:5337
1863598,chrUn_GL000218v1,90526,90622,+,163,A00132:417:HKVTWDSX3:4:1316:30246:8876


In [145]:


#============================================
rule sam_to_bam:
#============================================
    input:
        plus_sam=config['out_path']+"/{ID}/plus.5pfilt-tss.sam",
        minus_sam=config['out_path']+"/{ID}/minus.5pfilt-tss.sam"
    output:
        plus_bam=config['out_path']+"/{ID}/plus.5pfilt-tss.bam",
        minus_bam=config['out_path']+"/{ID}/minus.5pfilt-tss.bam"
    run:
        shell("samtools view -S -b {input.plus_sam} > {output.plus_bam}")
        shell("samtools view -S -b {input.minus_sam} > {output.minus_bam}")


#============================================
rule strand_merge_and_index:
#============================================
    input:
        plus_bam=config['out_path']+"/{ID}/plus.5pfilt-tss.bam",
        minus_bam=config['out_path']+"/{ID}/minus.5pfilt-tss.bam"
 
    output:
        merge_bam=config['out_path']+"/{ID}/5pfilt-tss.bam"
    run:
        shell("samtools merge -f {output.merge_bam} {input.plus_bam} {input.minus_bam}")
        shell("samtools index {output.merge_bam}")

(array([     0,      1,      2, ..., 816161, 816162, 816163]),
 array([     4,      6,      7, ..., 815750, 815758, 815751]))

In [146]:
np.setxor1d(np.arange(0,len(bam_pl)) , bam_ll[0].astype(int))

array([     0,      1,      2, ..., 816158, 816162, 816163])

In [147]:
pd.read_csv(meta_pl, sep='\t', header=None).iloc[np.setxor1d(np.arange(0,len(bam_pl)) , bam_ll[0].astype(int))]

,0,1,2,3,4,5,6,7,8
0,A00132:417:HKVTWDSX3:4:2462:26377:9095,355,chr1,26653,1,3S96M,=,26684,128
1,A00132:417:HKVTWDSX3:4:2634:11062:18991,99,chr1,26653,1,3S96M,=,26684,128
2,A00132:417:HKVTWDSX3:4:1144:25789:2253,99,chr1,26751,0,97M,=,26852,198
3,A00132:417:HKVTWDSX3:4:1303:19651:17769,355,chr1,26772,1,2S98M,=,26886,211
8,A00132:417:HKVTWDSX3:3:1358:11089:8672,355,chr1,90897,1,1S97M,=,90978,157
...,...,...,...,...,...,...,...,...,...
816156,A00132:417:HKVTWDSX3:3:2416:31430:11804,163,chr10,131906615,255,96M,=,131906678,163
816157,A00132:417:HKVTWDSX3:4:1648:4300:5556,163,chr10,131906615,255,96M,=,131906679,163
816158,A00132:417:HKVTWDSX3:4:1648:3296:15781,163,chr10,131906615,255,96M,=,131906680,163
816162,A00132:417:HKVTWDSX3:3:2677:10420:6668,163,chr10,131906637,255,97M,=,131906759,204


In [150]:
meta_pl

'plus.5pfilt.metadata.txt'

In [149]:
pd.read_csv(meta_pl, sep='\t', header=None)

,0,1,2,3,4,5,6,7,8
0,A00132:417:HKVTWDSX3:4:2462:26377:9095,355,chr1,26653,1,3S96M,=,26684,128
1,A00132:417:HKVTWDSX3:4:2634:11062:18991,99,chr1,26653,1,3S96M,=,26684,128
2,A00132:417:HKVTWDSX3:4:1144:25789:2253,99,chr1,26751,0,97M,=,26852,198
3,A00132:417:HKVTWDSX3:4:1303:19651:17769,355,chr1,26772,1,2S98M,=,26886,211
4,A00132:417:HKVTWDSX3:4:2414:18548:26694,355,chr1,26772,1,2S98M,=,26886,211
...,...,...,...,...,...,...,...,...,...
1413085,A00132:417:HKVTWDSX3:3:2133:27932:18333,99,chrUn_GL000218v1,77507,255,100M,=,77520,110
1413086,A00132:417:HKVTWDSX3:3:1456:15058:23093,355,chrUn_GL000218v1,90521,1,99M,=,90527,101
1413087,A00132:417:HKVTWDSX3:3:1456:15356:23735,355,chrUn_GL000218v1,90521,1,99M,=,90527,101
1413088,A00132:417:HKVTWDSX3:3:1456:15799:25285,355,chrUn_GL000218v1,90521,1,99M,=,90527,101


In [143]:
bam_pl

,Chromosome,Start,End,Strand,Flag,UMI
0,chr1,110980,111034,+,419,A00132:417:HKVTWDSX3:3:1175:10610:2675:read2
1,chr1,141566,141663,+,419,A00132:417:HKVTWDSX3:3:2152:8486:31845:read2
2,chr1,147525,147601,+,419,A00132:417:HKVTWDSX3:4:1577:3794:36182:read2
3,chr1,147525,147601,+,419,A00132:417:HKVTWDSX3:4:1577:3794:36182:read2
4,chr1,147641,147665,+,419,A00132:417:HKVTWDSX3:3:1213:7084:25692:read2
...,...,...,...,...,...,...
816159,chrUn_KI270742v1,78639,78734,+,163,A00132:417:HKVTWDSX3:3:2452:10384:13683:read2
816160,chrUn_KI270742v1,78639,78734,+,163,A00132:417:HKVTWDSX3:4:2425:6470:20087:read2
816161,chrUn_KI270742v1,78683,78778,+,163,A00132:417:HKVTWDSX3:4:1678:13485:13448:read2
816162,chrUn_KI270742v1,83574,83663,+,419,A00132:417:HKVTWDSX3:4:1369:32108:21089:read2


In [144]:
pl_umi

,0,1,2,3,4,5,6,7,8
0,A00132:417:HKVTWDSX3:4:2462:26377:9095,355,chr1,26653,1,3S96M,=,26684,128
1,A00132:417:HKVTWDSX3:4:2634:11062:18991,99,chr1,26653,1,3S96M,=,26684,128
2,A00132:417:HKVTWDSX3:4:1144:25789:2253,99,chr1,26751,0,97M,=,26852,198
3,A00132:417:HKVTWDSX3:4:1303:19651:17769,355,chr1,26772,1,2S98M,=,26886,211
8,A00132:417:HKVTWDSX3:3:1358:11089:8672,355,chr1,90897,1,1S97M,=,90978,157
...,...,...,...,...,...,...,...,...,...
816156,A00132:417:HKVTWDSX3:3:2416:31430:11804,163,chr10,131906615,255,96M,=,131906678,163
816157,A00132:417:HKVTWDSX3:4:1648:4300:5556,163,chr10,131906615,255,96M,=,131906679,163
816158,A00132:417:HKVTWDSX3:4:1648:3296:15781,163,chr10,131906615,255,96M,=,131906680,163
816162,A00132:417:HKVTWDSX3:3:2677:10420:6668,163,chr10,131906637,255,97M,=,131906759,204


In [ ]:
meta_pl_notin=config['out_path']+"/{ID}/plus.5pfilt-notin.metadata.txt",
meta_mi_notin=config['out_path']+"/{ID}/minus.5pfilt-notin.metadata.txt",
bam_ind=config['out_path']+"/{ID}/5pfilt-tss_splitstrand.index.npy"

In [136]:
meta_pl_notin="plus.5pfilt-notin.metadata.txt"
meta_mi_notin="minus.5pfilt-notin.metadata.txt"
pd.read_csv(meta_pl_notin, header=None)

,0
0,A00132:417:HKVTWDSX3:4:2462:26377:9095 355 chr...
1,A00132:417:HKVTWDSX3:4:2634:11062:18991 99 chr...
2,A00132:417:HKVTWDSX3:4:1144:25789:2253 99 chr1...
3,A00132:417:HKVTWDSX3:4:1303:19651:17769 355 ch...
4,A00132:417:HKVTWDSX3:3:1358:11089:8672 355 chr...
...,...
315073,A00132:417:HKVTWDSX3:3:2416:31430:11804 163 ch...
315074,A00132:417:HKVTWDSX3:4:1648:4300:5556 163 chr1...
315075,A00132:417:HKVTWDSX3:4:1648:3296:15781 163 chr...
315076,A00132:417:HKVTWDSX3:3:2677:10420:6668 163 chr...


In [130]:
mi_umi

,0,1,2,3,4,5,6,7,8
2,A00132:417:HKVTWDSX3:4:1654:27868:35853,403,chr1,99384,1,97M,=,99362,-119
3,A00132:417:HKVTWDSX3:4:2554:13847:36996,403,chr1,103957,3,97M,=,103932,-122
11,A00132:417:HKVTWDSX3:3:2658:25626:30968,403,chr1,121528,0,97M,=,121362,-263
12,A00132:417:HKVTWDSX3:4:1343:10719:27195,403,chr1,121528,0,97M,=,121362,-263
13,A00132:417:HKVTWDSX3:4:2277:2727:36714,403,chr1,121550,0,97M,=,121376,-271
...,...,...,...,...,...,...,...,...,...
727471,A00132:417:HKVTWDSX3:3:1143:22996:14231,83,chr10,81929062,255,1S97M2S,=,81929044,-115
727474,A00132:417:HKVTWDSX3:3:2443:24704:31939,83,chr10,81929091,255,3S97M,=,81929091,-97
727475,A00132:417:HKVTWDSX3:4:2542:7337:33207,83,chr10,81929138,255,99M,=,81929035,-202
727480,A00132:417:HKVTWDSX3:3:1251:28122:10050,83,chr10,81929156,255,95M5S,=,81928903,-348


In [131]:
ter.pysam_subset('plus.5pfilt-tss.bam')

[E::idx_find_and_load] Could not retrieve index file for 'plus.5pfilt-tss.bam'


,Chromosome,Start,End,Strand,Flag,UMI
0,chr1,110980,111034,+,419,A00132:417:HKVTWDSX3:3:1175:10610:2675:read2
1,chr1,141566,141663,+,419,A00132:417:HKVTWDSX3:3:2152:8486:31845:read2
2,chr1,287544,287598,+,419,A00132:417:HKVTWDSX3:3:1175:10610:2675:read2
3,chr1,467069,467166,+,419,A00132:417:HKVTWDSX3:3:2644:25789:34413:read2
4,chr1,467069,467166,+,419,A00132:417:HKVTWDSX3:3:2644:27190:34460:read2
...,...,...,...,...,...,...
632330,chrUn_KI270742v1,78639,78734,+,163,A00132:417:HKVTWDSX3:3:2452:10384:13683:read2
632331,chrUn_KI270742v1,78639,78734,+,163,A00132:417:HKVTWDSX3:4:2425:6470:20087:read2
632332,chrUn_KI270742v1,78683,78778,+,163,A00132:417:HKVTWDSX3:4:1678:13485:13448:read2
632333,chrUn_KI270742v1,83574,83663,+,419,A00132:417:HKVTWDSX3:4:1369:32108:21089:read2


In [243]:
out_bam[out_bam['Strand'] == '+']

,Chromosome,Start,End,Strand,Flag,UMI
41,chr1,31405,31502,+,419,A00132:417:HKVTWDSX3:3:2551:3721:30076
42,chr1,51632,51717,+,163,A00132:417:HKVTWDSX3:3:2105:30816:26929
57,chr1,111095,111166,+,419,A00132:417:HKVTWDSX3:4:2218:14488:27398
58,chr1,120887,165933,+,419,A00132:417:HKVTWDSX3:3:2323:15709:9032
90,chr1,124468,124545,+,419,A00132:417:HKVTWDSX3:4:1405:30906:9893
...,...,...,...,...,...,...
3614614,chrUn_KI270742v1,102192,102289,+,163,A00132:417:HKVTWDSX3:3:1238:30924:19382
3614656,chrUn_GL000218v1,72376,72471,+,163,A00132:417:HKVTWDSX3:4:2454:15148:28855
3614657,chrUn_GL000218v1,72377,72471,+,163,A00132:417:HKVTWDSX3:3:2304:13648:5337
3614715,chrUn_GL000218v1,90526,90622,+,163,A00132:417:HKVTWDSX3:4:1316:30246:8876


In [242]:
len(bam_pl), len(bam_ll[0]), len(bam_pl)-len(bam_ll[0]), len(bam_ll[0])+len(pl_umi)

(1863600, 320260, 1543340, 1863600)

In [132]:
ter.pysam_subset('minus.5pfilt-tss.bam')

[E::idx_find_and_load] Could not retrieve index file for 'minus.5pfilt-tss.bam'


,Chromosome,Start,End,Strand,Flag,UMI
0,chr1,87320,87417,-,147,A00132:417:HKVTWDSX3:4:1637:27877:29387:read2
1,chr1,99348,99444,-,403,A00132:417:HKVTWDSX3:4:1669:17020:27492:read2
2,chr1,121471,121567,-,403,A00132:417:HKVTWDSX3:3:1254:23249:17863:read2
3,chr1,121471,121567,-,403,A00132:417:HKVTWDSX3:3:1624:30508:5415:read2
4,chr1,121471,121567,-,403,A00132:417:HKVTWDSX3:3:2250:23972:36025:read2
...,...,...,...,...,...,...
565559,chrUn_GL000218v1,75145,75241,-,147,A00132:417:HKVTWDSX3:3:2335:17409:15295:read2
565560,chrUn_GL000218v1,75145,75241,-,403,A00132:417:HKVTWDSX3:3:2335:16441:15593:read2
565561,chrUn_GL000218v1,81131,81190,-,147,A00132:417:HKVTWDSX3:4:2238:17381:4852:read2
565562,chrUn_GL000218v1,148726,148822,-,403,A00132:417:HKVTWDSX3:3:1121:2130:15139:read2


In [120]:
len(bam_mi), len(bam_ll[1]), len(bam_mi)-len(bam_ll[1]), len(bam_ll[1])+len(mi_umi)

(727482, 424895, 302587, 727482)

In [133]:
out_bam[out_bam['Strand'] == '-']

,Chromosome,Start,End,Strand,Flag,UMI
0,chr1,87320,87417,-,147,A00132:417:HKVTWDSX3:4:1637:27877:29387:read2
1,chr1,99348,99444,-,403,A00132:417:HKVTWDSX3:4:1669:17020:27492:read2
3,chr1,121471,121567,-,403,A00132:417:HKVTWDSX3:3:1254:23249:17863:read2
4,chr1,121471,121567,-,403,A00132:417:HKVTWDSX3:3:1624:30508:5415:read2
5,chr1,121471,121567,-,403,A00132:417:HKVTWDSX3:3:2250:23972:36025:read2
...,...,...,...,...,...,...
1197894,chrUn_GL000218v1,75145,75241,-,147,A00132:417:HKVTWDSX3:3:2335:17409:15295:read2
1197895,chrUn_GL000218v1,75145,75241,-,403,A00132:417:HKVTWDSX3:3:2335:16441:15593:read2
1197896,chrUn_GL000218v1,81131,81190,-,147,A00132:417:HKVTWDSX3:4:2238:17381:4852:read2
1197897,chrUn_GL000218v1,148726,148822,-,403,A00132:417:HKVTWDSX3:3:1121:2130:15139:read2


In [113]:
mi_umi

,0,1,2,3,4,5,6,7,8
2,A00132:417:HKVTWDSX3:4:1654:27868:35853,403,chr1,99384,1,97M,=,99362,-119
3,A00132:417:HKVTWDSX3:4:2554:13847:36996,403,chr1,103957,3,97M,=,103932,-122
11,A00132:417:HKVTWDSX3:3:2658:25626:30968,403,chr1,121528,0,97M,=,121362,-263
12,A00132:417:HKVTWDSX3:4:1343:10719:27195,403,chr1,121528,0,97M,=,121362,-263
13,A00132:417:HKVTWDSX3:4:2277:2727:36714,403,chr1,121550,0,97M,=,121376,-271
...,...,...,...,...,...,...,...,...,...
727471,A00132:417:HKVTWDSX3:3:1143:22996:14231,83,chr10,81929062,255,1S97M2S,=,81929044,-115
727474,A00132:417:HKVTWDSX3:3:2443:24704:31939,83,chr10,81929091,255,3S97M,=,81929091,-97
727475,A00132:417:HKVTWDSX3:4:2542:7337:33207,83,chr10,81929138,255,99M,=,81929035,-202
727480,A00132:417:HKVTWDSX3:3:1251:28122:10050,83,chr10,81929156,255,95M5S,=,81928903,-348


In [24]:
os.chdir('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/Sample_5606_GLU/')
! ls

5pfilt-tss.bam			  minus.bam
5pfilt-tss.bam.bai		  plus.5pfilt.bam
5pfilt-tss_splitstrand.index.npy  plus.5pfilt.bam.bai
ATEM_counts.csv			  plus.5pfilt.metadata.txt
minus.5pfilt.bam		  plus.5pfilt-notin.metadata.txt
minus.5pfilt.bam.bai		  plus.5pfilt-tss.bam
minus.5pfilt.metadata.txt	  plus.5pfilt-tss.sam
minus.5pfilt-notin.metadata.txt   plus.5pfilt.withr1.bam
minus.5pfilt-tss.bam		  plus.5pfilt.withr1.bam.bai
minus.5pfilt-tss.sam		  plus.5pfilt_withsplice.bam
minus.5pfilt.withr1.bam		  plus.5pfilt_withsplice.bam.bai
minus.5pfilt.withr1.bam.bai	  plus.bam
minus.5pfilt_withsplice.bam	  splice_log.npy
minus.5pfilt_withsplice.bam.bai   total_reads.txt


In [27]:
path = '/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/Sample_5606_GLU/'


In [25]:
bam_ind='5pfilt-tss_splitstrand.index.npy'
merged_bam='5pfilt-tss.bam'
bam_pl='plus.5pfilt.bam'
bam_mi='minus.5pfilt.bam'

In [26]:
import numpy as np
import pandas as pd
import pysam

sys.path.insert(1, '/cndd3/dburrows/CODE/te_rna_ageing/')
import te_rna_f as te

def check(check_bool, check_bam, sign):
    bam_ = np.char.add((check_bam[check_bam['Strand'] == sign]['Start'].values).astype(str),  (check_bam[check_bam['Strand'] == sign]['Chromosome'].values).astype(str))
    bool_ = np.char.add(check_bool['Start'].values.astype(str), check_bool['Chromosome'].values.astype(str) )
    assert sum(np.in1d(bam_, bool_)) == len(bam_), 'Merged BAM file is missing some strand reads'
    assert sum(np.in1d(bool_, bam_)) == len(bool_), 'Merged BAM file is missing some strand reads'

In [245]:
merged_bam

'5pfilt-tss.bam'

In [34]:
#Final assertion
out_bam = te.pysam_subset(path+merged_bam) 
bam_ll_ = np.load(path+bam_ind, allow_pickle=True)
bam_pl = te.pysam_subset(path+bam_pl) 
bam_mi = te.pysam_subset(path+bam_mi) 

assert len(out_bam[out_bam['Strand'] == '+']) == len(bam_ll_[0]), 'Merged BAM file is missing some plus strand reads'
assert len(out_bam[out_bam['Strand'] == '-']) == len(bam_ll_[1]), 'Merged BAM file is missing some minus strand reads'
check(bam_pl.iloc[bam_ll_[0]],out_bam, '+')
check(bam_mi.iloc[bam_ll_[1]],out_bam, '-')
complete='All reads accounted for'
print(complete)
# np.save(snakemake.output.log_complete, complete)

assert len(out_bam[out_bam['Strand'] == '+']) == len(bam_ll[0]), 'Merged BAM file is missing some plus strand reads'
AssertionError: Merged BAM file is missing some plus strand reads

SyntaxError: invalid syntax (3010820285.py, line 17)

In [240]:
out_bam[out_bam['Strand'] == '+']

,Chromosome,Start,End,Strand,Flag,UMI
41,chr1,31405,31502,+,419,A00132:417:HKVTWDSX3:3:2551:3721:30076
42,chr1,51632,51717,+,163,A00132:417:HKVTWDSX3:3:2105:30816:26929
57,chr1,111095,111166,+,419,A00132:417:HKVTWDSX3:4:2218:14488:27398
58,chr1,120887,165933,+,419,A00132:417:HKVTWDSX3:3:2323:15709:9032
90,chr1,124468,124545,+,419,A00132:417:HKVTWDSX3:4:1405:30906:9893
...,...,...,...,...,...,...
3614614,chrUn_KI270742v1,102192,102289,+,163,A00132:417:HKVTWDSX3:3:1238:30924:19382
3614656,chrUn_GL000218v1,72376,72471,+,163,A00132:417:HKVTWDSX3:4:2454:15148:28855
3614657,chrUn_GL000218v1,72377,72471,+,163,A00132:417:HKVTWDSX3:3:2304:13648:5337
3614715,chrUn_GL000218v1,90526,90622,+,163,A00132:417:HKVTWDSX3:4:1316:30246:8876


In [241]:
len(bam_ll_[0])

320260

In [12]:
out_bam[out_bam['Strand'] == '+']

,Chromosome,Start,End,Strand,Flag,UMI
2,chr1,110980,111034,+,419,A00132:417:HKVTWDSX3:3:1175:10610:2675:read2
60,chr1,141566,141663,+,419,A00132:417:HKVTWDSX3:3:2152:8486:31845:read2
118,chr1,287544,287598,+,419,A00132:417:HKVTWDSX3:3:1175:10610:2675:read2
130,chr1,467069,467166,+,419,A00132:417:HKVTWDSX3:3:2644:25789:34413:read2
131,chr1,467069,467166,+,419,A00132:417:HKVTWDSX3:3:2644:27190:34460:read2
...,...,...,...,...,...,...
1197877,chrUn_KI270742v1,78639,78734,+,163,A00132:417:HKVTWDSX3:3:2452:10384:13683:read2
1197878,chrUn_KI270742v1,78639,78734,+,163,A00132:417:HKVTWDSX3:4:2425:6470:20087:read2
1197879,chrUn_KI270742v1,78683,78778,+,163,A00132:417:HKVTWDSX3:4:1678:13485:13448:read2
1197883,chrUn_KI270742v1,83574,83663,+,419,A00132:417:HKVTWDSX3:4:1369:32108:21089:read2


In [15]:
len(bam_ll[0])

501086

In [8]:
assert len(out_bam[out_bam['Strand'] == '-']) == len(bam_ll[1]), 'Merged BAM file is missing some minus strand reads'


AssertionError: Merged BAM file is missing some minus strand reads

In [ ]:
path = '/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/Sample_HCT16HCQA020_GLU/'


# Make new directories

In [39]:
curr = glob.glob('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM_old/Sample*')
curr = [os.path.basename(i) for i in curr]
os.chdir('/cndd/dburrows/DATA/te/rna/PE.counts/l1hs_count_bind/')
for c in curr:
    os.mkdir(c)

# in_path= "/cndd3/dburrows/DATA/te/rna/PE.bam/"
# out_path = "/cndd/dburrows/DATA/te/rna/PE.counts/TET/"

# samp_l = glob.glob(in_path+'/*Samp*')

# for s in samp_l:
#     #find base directory of s
#     base = os.path.basename(s)
#     if not os.path.exists(out_path+'/'+base):
#         os.mkdir(out_path+'/'+base)
# #5, 8, 9

In [93]:
#symlink files 
curr = glob.glob("/cndd/dburrows/DATA/te/rna/PE.bam_multi20/" + '*Samp*')
nex = glob.glob('/datasets/Public_Datasets/Dracheva_PsychEncode_development/processed/rna_seq/' + '*-*')

for c in curr:
    short = os.path.basename(c).split('_')[-1]
    prac = [i for i in nex if short in i]
    if len(prac) == 1:
         os.symlink(prac[0] + '/Aligned.sortedByCoord.out.bam', c + '/Aligned.sortedByCoord.out.bam')
    elif len(prac) < 1:
        short = short.upper()
        prac = [i for i in nex if short in i]
        if len(prac) == 1:
            os.symlink(prac[0] + '/Aligned.sortedByCoord.out.bam', c + '/Aligned.sortedByCoord.out.bam')
        else:
            if short == 'HCT16HECA028-GABA':
                short = 'HCK16HECA028-GABA'
            if short == 'HCT16HECA028-GLU':
                short = 'HCK16HECA028-GLU'
            prac = [i for i in nex if short in i]
            os.symlink(prac[0] + '/Aligned.sortedByCoord.out.bam', c + '/Aligned.sortedByCoord.out.bam')




In [105]:
#Delete all files in directory
os.chdir('/cndd3/dburrows/DATA/te/rna/CZI.counts/ATEM/pseudobulk')
curr = glob.glob('**')
for c in curr:
    #remove all contents of each directory
    shutil.rmtree(c)


# Check for log files

In [37]:
import glob
os.chdir('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/')
curr_ = glob.glob('*Sa*')
for c in curr_:
    os.chdir(c)
    atem = glob.glob('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/' + c + '/*ATEM*')
    log = glob.glob('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/' + c + '/*log*')
    if not atem or not log:
        print(c)
    if log:
        if 'All reads accounted for' not in np.load(log[0]):
            print(c)

    os.chdir('..')

In [28]:
import numpy as np
'All reads accounted for' in np.load(log[0])

True

# Save config files for snakemake

In [27]:
my_dict = {
    "in_path": "/cndd3/dburrows/DATA/te/rna/PE.bam", 
    "out_path": "/cndd/dburrows/DATA/te/rna/PE.counts/ATEM",
    "bed_plus_path": "/cndd3/dburrows/DATA/annotations/rmsk/rmsk.hg38.filt-5ptrim.plus.full.sort.bed",
    "bed_minus_path": "/cndd3/dburrows/DATA/annotations/rmsk/rmsk.hg38.filt-5ptrim.minus.full.sort.bed"
}

In [28]:
import os
import glob
os.chdir('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM')
curr_dict = my_dict.copy()
curr_dict['ID'] = glob.glob('*Samp*GABA*')
adm.make_config(curr_dict, '/cndd3/dburrows/CODE/te_rna_ageing/snakemake/workflow_PE-GABA.ATEM/config.json')

Write successful


In [22]:
curr_dict

{'in_path': '/cndd3/dburrows/DATA/te/rna/PE.bam',
 'out_path': '/cndd/dburrows/DATA/te/rna/PE.counts/ATEM',
 'bed_plus_path': '/cndd3/dburrows/DATA/annotations/rmsk/rmsk.hg38.filt-5ptrim.plus.full.sort.bed',
 'bed_minus_path': '/cndd3/dburrows/DATA/annotations/rmsk/rmsk.hg38.filt-5ptrim.minus.full.sort.bed',
 'ID': ['Sample_5077_GABA',
  'Sample_1275_GLU',
  'Sample_5086_GABA',
  'Sample_4379_GLU',
  'Sample_5570_GLU',
  'Sample_5918_GABA',
  'Sample_5744_GLU',
  'Sample_4337_GABA',
  'Sample_5184_GABA',
  'Sample_HCT15HBMA007_GABA',
  'Sample_4337_GLU',
  'Sample_198035-1765_GABA',
  'Sample_5977_GLU',
  'Sample_4428_GABA',
  'Sample_5643_GABA',
  'Sample_6285_GLU',
  'Sample_5976_GABA',
  'Sample_3739_GLU',
  'Sample_4425_GLU',
  'Sample_1277_GABA',
  'Sample_1134_GLU',
  'Sample_5570_GABA',
  'Sample_1133_GLU',
  'Sample_6285_GABA',
  'Sample_1823_GABA',
  'Sample_5161_GABA',
  'Sample_198035-1765_GLU',
  'Sample_179134-1401_GABA',
  'Sample_4321_GLU',
  'Sample_5643_GLU',
  'Sample

In [ ]:
"bed_minus_path": "/cndd3/dburrows/DATA/annotations/rmsk/rmsk.hg38.filt-5ptrim.minus.full.sort.bed", "ID": ["Sample_5086-GLU", "Sample_4369-GLU", "Sample_5309-GLU", "Sample_4411-GLU", "Sample_5871-GLU", "Sample_5936-GLU", "Sample_1241-GLU", "Sample_5387-GLU", "Sample_5976-GLU", "Sample_4725-GLU", "Sample_1539-GLU", "Sample_Hct16HCQA020-GLU", "Sample_5643-GLU", "Sample_4321-GLU", "Sample_1133-GLU", "Sample_1134-GLU", "Sample_4425-GLU", "Sample_3739-GLU", "Sample_6285-GLU", "Sample_5977-GLU", "Sample_4337-GLU", "Sample_HctZZNA014-GLU", "Sample_5744-GLU", "Sample_5570-GLU", "Sample_4379-GLU", "Sample_Hct15HBMA007-GLU", "Sample_1275-GLU", "Sample_Hct17HEIA010-GLU", "Sample_HctYPA008-GLU", "Sample_6007-GLU", "Sample_1535-GLU", "Sample_5326-GLU", "Sample_6032-GLU", "Sample_198034-1435-GLU", "Sample_5184-GLU", "Sample_5293-GLU", "Sample_1823-GLU", "Sample_Hct16HECA028-GLU", "Sample_1105-GLU", "Sample_4414-GLU", "Sample_179134-1401-GLU", "Sample_5401-GLU", "Sample_4545-GLU", "Sample_5787-GLU", "Sample_1648-GLU", "Sample_5446-GLU", "Sample_4428-GLU", "Sample_Hct15HAWA031-GLU", "Sample_Hct15HBNA032-GLU", "Sample_5077-GLU", "Sample_1277-GLU", "Sample_5451-GLU", "Sample_4332-GLU", "Sample_198035-1765-GLU", "Sample_5161-GLU", "Sample_5617-GLU", "Sample_5918-GLU", "Sample_5606-GLU", "Sample_1848-GLU"]}

In [98]:
curr_dict

{'in_path': '/cndd/dburrows/DATA/te/rna/PE.bam_multi20',
 'out_path': '/cndd/dburrows/DATA/te/rna/PE.counts/RNA-mut',
 'ID': ['Sample_5570-GABA',
  'Sample_5086-GLU',
  'Sample_1823-GABA',
  'Sample_6285-GABA',
  'Sample_1277-GABA',
  'Sample_5976-GABA',
  'Sample_5161-GABA',
  'Sample_4411-GABA',
  'Sample_4369-GLU',
  'Sample_5326-GABA',
  'Sample_HctYPA008-GABA',
  'Sample_5309-GLU',
  'Sample_4411-GLU',
  'Sample_5871-GLU',
  'Sample_5077-GABA',
  'Sample_198034-1435-GABA',
  'Sample_5086-GABA',
  'Sample_Hct15HBNA032-GABA',
  'Sample_5936-GLU',
  'Sample_4337-GABA',
  'Sample_1241-GLU',
  'Sample_5184-GABA',
  'Sample_5918-GABA',
  'Sample_5643-GABA',
  'Sample_5387-GLU',
  'Sample_5976-GLU',
  'Sample_4725-GLU',
  'Sample_4428-GABA',
  'Sample_1539-GLU',
  'Sample_4332-GABA',
  'Sample_4379-GABA',
  'Sample_1848-GABA',
  'Sample_179134-1401-GABA',
  'Sample_Hct16HCQA020-GLU',
  'Sample_5643-GLU',
  'Sample_4321-GLU',
  'Sample_5744-GABA',
  'Sample_5401-GABA',
  'Sample_5293-GABA

In [38]:
[(curr[i], p) for i in range(len(curr)) if p in curr[i]]

[('5451-GABA_ATGGCATG-GGTACCTT_HKVTWDSX3', '5451-GABA')]

In [42]:
prac = [i.split('_')[-1] for i in curr_dict['ID']]
os.chdir('/datasets/Public_Datasets/Dracheva_PsychEncode_development/processed/rna_seq/')
curr = glob.glob('*-*')
for p in prac:
    print(p)
    print([(curr[i], p) for i in range(len(curr)) if p in curr[i]])

5570-GABA
[('5570-GABA_TCTCTACT-GAACCGCG_HKVTWDSX3', '5570-GABA')]
5086-GLU
[('5086-GLU_TCTGTTGG-CCATTCGA_HKVTWDSX3', '5086-GLU')]
1823-GABA
[('1823-GABA_GATCTATC-AGCCTCAT_HKVTWDSX3', '1823-GABA')]
6285-GABA
[('6285-GABA_ACTAAGAT-CCGCGGTT_HKVTWDSX3', '6285-GABA')]
1277-GABA
[('1277-GABA_AAGATACT-ATGTAAGT_HKVTWDSX3', '1277-GABA')]
5976-GABA
[('5976-GABA_GCCACAGG-ATGGCATG_HKVTWDSX3', '5976-GABA')]
5161-GABA
[('5161-GABA_AGGTTATA-CGGAACTG_HKVTWDSX3', '5161-GABA')]
4411-GABA
[('4411-GABA_CAATTAAC-ATATCTCG_HKVTWDSX3', '4411-GABA')]
4369-GLU
[('4369-GLU_TAAGGTCA-CTACGACA_HKVTWDSX3', '4369-GLU')]
5326-GABA
[('5326-GABA_ATGTAAGT-CATAGAGT_HKVTWDSX3', '5326-GABA')]
HctYPA008-GABA
[]
5309-GLU
[('5309-GLU_GCGCAAGC-CGGCGTGA_HKVTWDSX3', '5309-GLU')]
4411-GLU
[('4411-GLU_TGGCCGGT-GCGCTCTA_HKVTWDSX3', '4411-GLU')]
5871-GLU
[('5871-GLU_ACGCACCT-CCTTCACC_HKVTWDSX3', '5871-GLU')]
5077-GABA
[('5077-GABA_ACACTAAG-ATATGGAT_HKVTWDSX3', '5077-GABA')]
198034-1435-GABA
[('198034-1435-GABA_TATCGCAC-ACACTAAG_HKVT

In [18]:
curr_dict

{'in_path': '/cndd3/dburrows/DATA/te/rna/PE.bam',
 'out_path': '/cndd/dburrows/DATA/te/rna/PE.counts/RNA-mut',
 'ID': ['Sample_5570-GABA',
  'Sample_5086-GLU',
  'Sample_1823-GABA',
  'Sample_6285-GABA',
  'Sample_1277-GABA',
  'Sample_5976-GABA',
  'Sample_5161-GABA',
  'Sample_4411-GABA',
  'Sample_4369-GLU',
  'Sample_5326-GABA',
  'Sample_HctYPA008-GABA',
  'Sample_5309-GLU',
  'Sample_4411-GLU',
  'Sample_5871-GLU',
  'Sample_5077-GABA',
  'Sample_198034-1435-GABA',
  'Sample_5086-GABA',
  'Sample_Hct15HBNA032-GABA',
  'Sample_5936-GLU',
  'Sample_4337-GABA',
  'Sample_1241-GLU',
  'Sample_5184-GABA',
  'Sample_5918-GABA',
  'Sample_5643-GABA',
  'Sample_5387-GLU',
  'Sample_5976-GLU',
  'Sample_4725-GLU',
  'Sample_4428-GABA',
  'Sample_1539-GLU',
  'Sample_4332-GABA',
  'Sample_4379-GABA',
  'Sample_1848-GABA',
  'Sample_179134-1401-GABA',
  'Sample_Hct16HCQA020-GLU',
  'Sample_5643-GLU',
  'Sample_4321-GLU',
  'Sample_5744-GABA',
  'Sample_5401-GABA',
  'Sample_5293-GABA',
  'S

In [34]:
os.chdir('/cndd3/mcuoco/projects/czi_diversity_rnate/results/merge_bam/')
curr_l = glob.glob('**')
bam_ll = []
for curr in curr_l:
    bam_l = glob.glob(curr+'/*.bam')
    print(len(bam_l))
    bam_ll = np.append(bam_ll, bam_l)

19
19
19
19
18
19
19
19
19
19
19
19
19
19
18
19
18
19
19
19
19


In [8]:
import os
import glob
os.chdir('/cndd3/dburrows/DATA/te/rna/PE.bam')
l = glob.glob('*Samp**')
for n in l:
    os.mkdir('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/' + n)

In [6]:
l

['Sample_5570-GABA',
 'Sample_5086-GLU',
 'Sample_1823-GABA',
 'Sample_6285-GABA',
 'Sample_1277-GABA',
 'Sample_5976-GABA',
 'Sample_5161-GABA',
 'Sample_4411-GABA',
 'Sample_4369-GLU',
 'Sample_5326-GABA',
 'Sample_HctYPA008-GABA',
 'Sample_5309-GLU',
 'Sample_4411-GLU',
 'Sample_5871-GLU',
 'Sample_5077-GABA',
 'Sample_198034-1435-GABA',
 'Sample_5086-GABA',
 'Sample_Hct15HBNA032-GABA',
 'Sample_5936-GLU',
 'Sample_4337-GABA',
 'Sample_1241-GLU',
 'Sample_5184-GABA',
 'Sample_5918-GABA',
 'Sample_5643-GABA',
 'Sample_5387-GLU',
 'Sample_5976-GLU',
 'Sample_4725-GLU',
 'Sample_4428-GABA',
 'Sample_1539-GLU',
 'Sample_4332-GABA',
 'Sample_4379-GABA',
 'Sample_1848-GABA',
 'Sample_179134-1401-GABA',
 'Sample_Hct16HCQA020-GLU',
 'Sample_5643-GLU',
 'Sample_4321-GLU',
 'Sample_5744-GABA',
 'Sample_5401-GABA',
 'Sample_5293-GABA',
 'Sample_1133-GLU',
 'Sample_1134-GLU',
 'Sample_4425-GLU',
 'Sample_3739-GLU',
 'Sample_6285-GLU',
 'Sample_6007-GABA',
 'Sample_4425-GABA',
 'Sample_1133-GABA

In [2]:
import pandas as pd
bed_pl = pd.read_csv("/cndd3/dburrows/DATA/te/gtf/bed/rmsk.hg38.filt-5ptrim.plus.sort.bed", sep='\t', header=None)



In [4]:
bed_pl

,0,1,2,3,4,5,6,7
0,chr1,24038,24138,+,MIR,MIR,SINE,211
1,chr1,26741,26841,+,AluSp,Alu,SINE,263
2,chr1,31386,31486,+,AluJo,Alu,SINE,297
3,chr1,32791,32891,+,MIR,MIR,SINE,214
4,chr1,39574,39674,+,AluSx,Alu,SINE,300
...,...,...,...,...,...,...,...,...
548906,chrY,57200662,57200762,+,AluSg7,Alu,SINE,308
548907,chrY,57201438,57201538,+,AluSp,Alu,SINE,305
548908,chrY,57202215,57202315,+,AluSz,Alu,SINE,278
548909,chrY,57202519,57202619,+,AluSc,Alu,SINE,308


In [7]:
bed_pl.columns =['Chromosome', 'Start', 'End', 'Strand', 'gene_id', 'family_id', 'class_id', 'length']
bed_pl

,Chromosome,Start,End,Strand,gene_id,family_id,class_id,length
0,chr1,24038,24138,+,MIR,MIR,SINE,211
1,chr1,26741,26841,+,AluSp,Alu,SINE,263
2,chr1,31386,31486,+,AluJo,Alu,SINE,297
3,chr1,32791,32891,+,MIR,MIR,SINE,214
4,chr1,39574,39674,+,AluSx,Alu,SINE,300
...,...,...,...,...,...,...,...,...
548906,chrY,57200662,57200762,+,AluSg7,Alu,SINE,308
548907,chrY,57201438,57201538,+,AluSp,Alu,SINE,305
548908,chrY,57202215,57202315,+,AluSz,Alu,SINE,278
548909,chrY,57202519,57202619,+,AluSc,Alu,SINE,308


In [8]:
#drop last columns
bed_pl = bed_pl.drop(columns=['length'])
bed_pl

,Chromosome,Start,End,Strand,gene_id,family_id,class_id
0,chr1,24038,24138,+,MIR,MIR,SINE
1,chr1,26741,26841,+,AluSp,Alu,SINE
2,chr1,31386,31486,+,AluJo,Alu,SINE
3,chr1,32791,32891,+,MIR,MIR,SINE
4,chr1,39574,39674,+,AluSx,Alu,SINE
...,...,...,...,...,...,...,...
548906,chrY,57200662,57200762,+,AluSg7,Alu,SINE
548907,chrY,57201438,57201538,+,AluSp,Alu,SINE
548908,chrY,57202215,57202315,+,AluSz,Alu,SINE
548909,chrY,57202519,57202619,+,AluSc,Alu,SINE


In [1]:
#Check that all files are present
import glob
import os

full_l = os.listdir('/cndd2/Public_Datasets/Dracheva_PsychEncode_development/raw_May2022/') 
os.chdir('/cndd3/dburrows/DATA/te/rna/PE.bam/')
check_l = glob.glob('*Samp*')

import numpy as np
not_int = np.setxor1d(check_l, full_l)
assert len(full_l) == len(check_l)
assert len(not_int) == 0

In [ ]:
STAR --genomeDir /cndd2/jchien/iGenome/STAR_gencode_v37 --readFilesIn  R1-merge.fastq.gz R1-merge.fastq.gz --outSAMunmapped None --outFilterType BySJout --outSAMattributes All --outFilterMultimapNmax 100 --outFilterMismatchNmax 999 --outFilterMismatchNoverReadLmax 0.04 --alignIntronMin 20 --alignIntronMax 1000000 --alignMatesGapMax 1000000 --alignSJoverhangMin 8 --alignSJDBoverhangMin 1 --sjdbScore 1 --readFilesCommand zcat --runThreadN 16--genomeLoad LoadAndKeep --limitBAMsortRAM 10000000000 --outSAMtype BAM SortedByCoordinate --quantMode TranscriptomeSAM GeneCounts --winAnchorMultimapNmax 200 --outMultimapperOrder Random --outSAMmultNmax -1

In [76]:
# Check which files went to completion
import glob
os.chdir('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/')
curr_ = glob.glob('*Sa*')
for c in curr_:
    os.chdir(c)
    atem = glob.glob('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/' + c + '/*ATEM*')
    log = glob.glob('/cndd/dburrows/DATA/te/rna/PE.counts/ATEM/' + c + '/*log*')
    if not atem or not log:
        print(c)
    if log:
        if 'All reads accounted for' not in np.load(log[0]):
            print(c)

    os.chdir('..')